In [ ]:
'''
READ ME:

This first block of code will not work in an arcgis environment but should work in an online jupyter notebooks environment and pulls
data from the metro transit realtime GTFS feed using information on vehicle location.
'''

import csv
import datetime
import time
import requests
from google.transit import gtfs_realtime_pb2

# Open a CSV file for writing
csv_filename = r'gtfs_data_test_1.csv'
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(['id', 'latitude', 'longitude', 'timestamp', 'trip_id', 'start_date'])

    # Set the end count to limit the number of loops
    end_count = 0

    # Continuous loop for 3 cycles
    while end_count < 3:
        try:
            # Request GTFS feed
            response = requests.get('https://svc.metrotransit.org/mtgtfs/vehiclepositions.pb')
            feed = gtfs_realtime_pb2.FeedMessage()
            feed.ParseFromString(response.content)

            # Process each entity in the feed
            for entity in feed.entity:
                if entity.HasField('vehicle'):
                    # Extract information
                    vehicle_id = entity.vehicle.vehicle.id
                    latitude = entity.vehicle.position.latitude
                    longitude = entity.vehicle.position.longitude
                    timestamp = entity.vehicle.timestamp
                    trip_id = entity.vehicle.trip.trip_id
                    start_date = entity.vehicle.trip.start_date
                    
                    # Convert timestamp from seconds to formatted date string
                    timestamp_dt = datetime.datetime.fromtimestamp(timestamp)
                    timestamp_str = timestamp_dt.strftime('%Y-%m-%d %H:%M:%S')

                    # Write the information to the CSV file only if the data is clean
                    if latitude != 0.0 and longitude != 0.0 and timestamp != 0: 
                        writer.writerow([vehicle_id, latitude, longitude, timestamp_str, trip_id, start_date])

        except Exception as e:
            print("Error:", e)

        # Wait for 15 seconds before requesting the feed again
        time.sleep(15)
        end_count = end_count + 1

print("Data collection complete. CSV file saved as:", csv_filename)

In [5]:
import pandas as pd
import datetime

df = pd.read_csv(r'C:\School\GIS 5572\Local\Lab 2\project data\gtfs_data_test_1.csv')
df = df.drop_duplicates()
df['simple_time'] = pd.to_datetime(df['timestamp'])
df['simple_time'] = df['simple_time'].dt.strftime('%H:%M')
df.to_csv(r'C:\School\GIS 5572\Local\Lab 2\project data\gtfs_data_test_1.csv', index=False)

In [7]:
arcpy.env.workspace = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\Test Project.gdb'
spatial_ref = arcpy.SpatialReference(4326)
out_name = "bus_location_points"
table_in = r'C:\School\GIS 5572\Local\Lab 2\project data\gtfs_data_test_1.csv'
arcpy.management.XYTableToPoint(table_in, out_name, "longitude", "latitude", None, spatial_ref)

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\Test Project.gdb\\bus_location_points'>

In [8]:
arcpy.env.workspace = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\PostgreSQL-35-gis5572(postgres).sde'
spatial_ref = arcpy.SpatialReference(4326)
out_name = "bus_location_points"
geometry_type = "POINT"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, out_name, geometry_type, spatial_reference = spatial_ref)

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\PostgreSQL-35-gis5572(postgres).sde\\bus_location_points'>

In [9]:
in_features = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\Test Project.gdb\bus_location_points'
out_features = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\PostgreSQL-35-gis5572(postgres).sde\bus_location_points'
arcpy.management.CopyFeatures(in_features, out_features)

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\PostgreSQL-35-gis5572(postgres).sde\\gis5572.postgres.bus_location_points'>